# Predict radical

In [1]:
import sys
sys.path.insert(0,"/home/gridsan/hwpang/Software/RMG-Py/")
sys.path.insert(0,"..")

import os
import json
import pandas as pd

from rmgpy import settings
from rmgpy.data.thermo import ThermoDatabase, ThermoGroups

from tree.thermo import ThermoGroups as SIDTThermoGroups
from tree.utils import make_mol, generate_thermo
from tree.parameters import Ts

# Load test data

In [2]:
hbi_unc_df = pd.read_csv("../data/hbi_unc.csv")

In [3]:
with open("../data/splits/random.json", "r") as f:
    train_inds, test_inds = json.load(f)

In [4]:
test_df = hbi_unc_df.loc[test_inds, :]
test_df

,radical_smiles,resonance_radical_smiles,resonance_radical_num_rotatable_bonds,radical_H298 (kcal/mol),radical_Sint298 (cal/mol/K),radical_source,radical_level_of_theory,radical_Cp300 (cal/mol/K),radical_Cp400 (cal/mol/K),radical_Cp500 (cal/mol/K),...,unc_closed_shell_Cp1500 (cal/mol/K),unc_HBI_H298 (kcal/mol),unc_HBI_Sint298 (cal/mol/K),unc_HBI_Cp300 (cal/mol/K),unc_HBI_Cp400 (cal/mol/K),unc_HBI_Cp500 (cal/mol/K),unc_HBI_Cp600 (cal/mol/K),unc_HBI_Cp800 (cal/mol/K),unc_HBI_Cp1000 (cal/mol/K),unc_HBI_Cp1500 (cal/mol/K)
14,[O]C(=O)OC(=O)O,[O]C(=O)OC(=O)O,0,-164.838453,87.670293,dong_pio_liang.py,CBS-QB3,23.387634,26.613081,29.284902,...,1.0,2.4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
15,CC(=O)COC(O[O])OC(C)=O,CC(=O)COC(O[O])OC(C)=O,7,-164.222931,134.520962,dong_pio_liang.py,CBS-QB3,51.800544,60.936453,68.795453,...,2.0,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
18,[O]OC(=O)OC(=O)O,[O]OC(=O)OC(=O)O,0,-155.105883,93.562651,dong_pio_liang.py,CBS-QB3,29.543449,33.217979,35.694866,...,1.0,2.4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
25,C[C]1OC(=O)C(O)(C=O)O1,C[C]1OC(=O)C(O)(C=O)O1,3,-151.205886,98.996040,dong_pio_liang.py,CBS-QB3,38.751751,46.650997,52.369635,...,2.0,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
36,CC1(OO)COC(O[O])C(=O)O1,CC1(OO)COC(O[O])C(=O)O1,4,-137.513199,113.097378,dong_pio_liang.py,CBS-QB3,46.328845,56.831030,65.250112,...,2.0,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2771,[CH]1C=C1,[CH]1C=C1,0,117.221627,61.749708,dong_pio_liang.py,CBS-QB3,12.226569,15.047122,17.464371,...,1.0,2.4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2779,[CH]=CC=C=C,[CH]=CC=C=C,1,121.550393,73.677623,dong_pio_liang.py,CBS-QB3,22.329417,27.626795,31.770255,...,1.0,2.4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2793,C1=C=CC[CH]C=1,C1=C=CC[CH]C=1,0,139.224273,73.089716,dong_pio_liang.py,CBS-QB3,21.439563,27.184618,32.155575,...,2.0,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2794,C1=C=CC[CH]C=1,[C]1=C=CCC=C1,0,139.224273,73.089716,dong_pio_liang.py,CBS-QB3,21.439563,27.184618,32.155575,...,1.0,2.4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [5]:
radical_smis = test_df["radical_smiles"].drop_duplicates(keep="first")

# Load thermo database

In [6]:
# load thermo database

thermo_database = ThermoDatabase()
thermo_database.load_groups(os.path.join(settings["database.directory"], "thermo", "groups"))

INFO:root:Loading thermodynamics group database from /home/gridsan/hwpang/Software/RMG-database/input/thermo/groups...


In [7]:
model_dir = "split-random_run-aleatoric_prepruning-upper_bound/frac-1.0/"
sidt_tree = SIDTThermoGroups().load(f"../models/{model_dir}/tree.py", thermo_database.local_context, thermo_database.global_context)

In [8]:
thermo_database.groups["radical"] = sidt_tree

# Predict with SIDT tree

In [9]:
generate_thermo(thermo_database, "[O]C(=O)OC(O)(O)O", resonance=True) # we want to know the best GAV estimation



/home/gridsan/hwpang/Software/RMG-Py/rmgpy/rmg/reactors.py:52: RuntimeWarning: Unable to import Julia dependencies, original error: No module named 'julia'
  warnings.warn("Unable to import Julia dependencies, original error: " + str(e), RuntimeWarning)


(-223.51759082217976,
 92.43938152542268,
 array([33.92687358, 38.38767675, 41.50849629, 44.06238983, 47.77973491,
        50.08932478, 52.51017141]),
 95.99997759082218,
 'Thermo group additivity estimation: group(O2s-Cs(Cds-O2d)) + group(O2s-CsH) + group(O2s-CsH) + group(O2s-CsH) + group(O2s-(Cds-O2d)H) + group(Cs-OsOsOsOs) + group(Cds-OdOsOs) + (RJ1_N-1R-inRing_N-1R->C_Ext-1HNO-R_2R!H->C_Ext-2C-R_Sp-3R!H=2C_N-3R!H->C_Ext-2C-R_Ext-4R!H-R_Ext-5R!H-R_Ext-5R!H-R)')

In [10]:
thermos = [generate_thermo(thermo_database, smi, resonance=True) for smi in radical_smis]

In [11]:
test_result_df = pd.DataFrame()
test_result_df["radical_smiles"] = radical_smis
test_result_df["H298 (kcal/mol)"] = [thermo[0] for thermo in thermos]
test_result_df["S298 (cal/mol/K)"] = [thermo[1] for thermo in thermos]
for i, T in enumerate(Ts):
    test_result_df[f"Cp{T} (cal/mol/K)"] = [thermo[2][i] for thermo in thermos]
test_result_df["comment"] = [thermo[4] for thermo in thermos]

In [12]:
test_result_df.to_csv(f"../models/{model_dir}/test_radical.csv", index=False)